In [1]:
import torch

In [2]:
from mife.single.selective.ddh import FeDDH

In [3]:
if __name__ == '__main__':
    n = 10
    x = [i for i in range(n)]
    y = [i + 10 for i in range(n)]
    key = FeDDH.generate(n)
    c = FeDDH.encrypt(x, key)
    sk = FeDDH.keygen(y, key)
    m = FeDDH.decrypt(c, key.get_public_key(), sk, (0, 1000))

In [4]:
import numpy as np
from mife.multiclient.rom.ddh import FeDDHMultiClient
import json

# Parameters
n, m = 3, 5
tag = b"testingtag123"

# 1) Build x and y as NumPy arrays via broadcasting
i = np.arange(n).reshape(n, 1)   # shape (n, 1)
j = np.arange(m)                 # shape (m,)
x = i + j                        # shape (n, m): each row i has values i + j
y = i - j + 10                   # shape (n, m): each row i has values i - j + 10

# 2) Generate the multi-client key
key = FeDDHMultiClient.generate(n, m)

# 3) Encrypt each row of x
cs = []
for idx, row in enumerate(x):
    enc_key = key.get_enc_key(idx)
    # The library may accept a NumPy array directly; if not, convert to list:
    row_data = row.tolist()
    cs.append(FeDDHMultiClient.encrypt(row_data, tag, enc_key))

# 4) Generate the secret key on y
# If keygen expects a list of lists, convert y to nested Python lists:
sk = FeDDHMultiClient.keygen(y.tolist(), key)

# 5) Serialize and print everything
enc_keys_exported = [key.get_enc_key(i).export() for i in range(n)]
print("enc_key =", json.dumps(enc_keys_exported))
print("msk =", json.dumps(key.export()))
print("ct =", json.dumps([ct.export() for ct in cs]))
print("secret_key =", json.dumps(sk.export()))
print("pub_key =", json.dumps(key.get_public_key().export()))


enc_key = [{"g": {"val": 63536483878621043970475411345584093935762373157525764224938730624935043406348610114560867589210704863512068258870402709942423806816503970569622286105330266147450924200047870973653677772343134654313802444910426223283743714830626334833272472028848032385277236404665861866753994499443240841974008303865823948629}, "hash": {"type": "default", "maximum_bit": 1024}, "enc_key": [[16990330954053860186554835536122327379897409396342662787863765597139754689631445919022738800243147847637309757265176004532561885587035600370165289301516131197822905570298330657890992673498150680208127418367085086387653099938562941557066115974872114568973037338036668640804224541403298938207931228260797170285, 433323394828700786155389486942106375248809336085822698828861136806148721589424772200090730212774100410304015875010397029456542101397263721132653088870486896269172177448077744949185300093200003421278692101126852648859613282586937667653490118778715143541587979677795660992752661625617887812552

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
import torchvision
from torchvision import datasets, transforms
from mife.multiclient.rom.ddh import FeDDHMultiClient
import numpy as np

# 1) Dataset: MNIST 0 vs 1
transform = transforms.Compose([
    transforms.ToTensor(),  # Scales to [0,1]
])
full_train = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
# Filter for labels 0 or 1
idxs = [i for i, (_, label) in enumerate(full_train) if label in (0, 1)]
train_subset = Subset(full_train, idxs)
loader = DataLoader(train_subset, batch_size=64, shuffle=True)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Verify versions
print("torch:", torch.__version__, "CUDA available?", torch.cuda.is_available())
print("torchvision:", torchvision.__version__)

# 2) Define CNN suitable for MNIST
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=3, stride=1, padding=1),  # [1, 28, 28] -> [8, 28, 28]
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)  # [8, 28, 28] -> [8, 14, 14]
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1),  # [8, 14, 14] -> [16, 14, 14]
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)  # [16, 14, 14] -> [16, 7, 7]
        )
        self.flatten = nn.Flatten()
        self.feat = nn.Linear(16 * 7 * 7, 16)  # [16*7*7=784] -> [16]
        self.fc = nn.Linear(16, 1)  # [16] -> [1]
        self.act = nn.Sigmoid()
        # Initialize weights
        nn.init.xavier_uniform_(self.feat.weight)
        nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.flatten(x)  # [batch, 16*7*7]
        feats = torch.relu(self.feat(x))  # [batch, 16]
        out = self.act(self.fc(feats))  # [batch, 1]
        return out, feats  # Return (prediction, features)

model = CNN().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 3) Train briefly
model.train()
for epoch in range(10):
    total_loss = 0.0
    for imgs, labels in loader:
        imgs = imgs.to(device)
        labels = labels.float().unsqueeze(1).to(device)
        optimizer.zero_grad()
        preds, _ = model(imgs)
        loss = criterion(preds, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1} loss: {total_loss/len(loader):.4f}")

# 4) Extract features
model.eval()
features = []
with torch.no_grad():
    for imgs, _ in DataLoader(train_subset, batch_size=256):
        imgs = imgs.to(device)
        _, feats = model(imgs)
        features.append(feats.cpu().numpy())
X_feat = np.vstack(features)  # Shape (n, 16)
n, m = X_feat.shape

# 5) Get final-layer weight vector
y_vec = model.fc.weight.detach().cpu().numpy().reshape(-1)  # Use fc instead of out

# 6) Quantize to integers
scale = 1000
X_int = np.round(X_feat * scale).astype(int)
y_int = np.round(y_vec * scale).astype(int)

# 7) Run FeDDHMultiClient
tag = b"pytorch-cnn-fe"
fe = FeDDHMultiClient.generate(n, m)

# Encrypt each feature vector
cs = [
    FeDDHMultiClient.encrypt(
        X_int[i].tolist(),
        tag,
        fe.get_enc_key(i)
    )
    for i in range(n)
]

# Create a matrix by repeating y_int n times to match expected shape (n, m)
y_mat = np.tile(y_int, (n, 1))  # Shape: (n, m) = (12665, 16)

# Generate secret key for decryption
sk = FeDDHMultiClient.keygen(y_mat.tolist(), fe)

# Decrypt to compute Σ⟨xᵢ, y⟩
result = FeDDHMultiClient.decrypt(
    cs, tag,
    fe.get_public_key(),
    sk,
    (0, 10**12)
)

# Validate result by computing unencrypted inner product
expected_result = np.sum(X_int @ y_int)
print("Decrypted Σ⟨xᵢ, y⟩ =", result)
print("Expected Σ⟨xᵢ, y⟩ =", expected_result)

torch: 2.4.0+cu121 CUDA available? True
torchvision: 0.19.0+cu121
Epoch 1 loss: 0.0628
Epoch 2 loss: 0.0047
Epoch 3 loss: 0.0039
Epoch 4 loss: 0.0025
Epoch 5 loss: 0.0026
Epoch 6 loss: 0.0017
Epoch 7 loss: 0.0017
Epoch 8 loss: 0.0017
Epoch 9 loss: 0.0020
Epoch 10 loss: 0.0018


KeyboardInterrupt: 